In [1]:
import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
#Yashwanth key
'''consumer_key = "CSuIiOnVjfkIVlwoOt8TmrGW7"
consumer_key_secret = "wz0VqziOsFNr1vaz1RY3XySNHajIeW7D2SKdyMP76Z89rJuHvd"
access_token = "1457745168690450437-7dnKuIzrBMIxTJSqQ0eSfaNycyp7BV"
access_token_secret = "mfkismanuwheyKmCsXCbFIVLNibt2MW8CZas3qfqFPken"'''
#parth key:
consumer_key = "FVFGY8gFJGp77OOM6Do5OprZ4"
consumer_key_secret = "2utlJGqBy3lObyRPRSxMiFUWOoQNjgCmd5Ia4E3XlYO0YQyCzY"
access_token = "1452533975180738562-bfBTDql81sBKYmJfVojaQY16Z35M53"
access_token_secret = "xPWXQNu9ElozB7RcXRS4RBQLdQHprkQP0ZDBjUHgJLFnI"

In [3]:
auth = tweepy .OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
def cleanText(text):
    text = re.sub(r"@[A-Za-z0-9]+", '', text) #removed @mentions
    text = re.sub(r'#', '', text) #removing # symbol
    text = re.sub(r'RT[\s]+', '', text) #removing retweets RT
    text = re.sub(r'https?:\/\/\S+', '', text) #removes the hyperlink

    return text

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getAnalysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

In [5]:
def get_keyword_tweets(keyword, count):
    items = api.search_30_day(label='development', query=keyword, maxResults= count)
    return items

In [6]:
def get_new_table(stockname):
    answer = get_keyword_tweets(stockname, 100)
    df = pd.DataFrame({"tweet": [tweet.text for tweet in answer], "language":[tweet.lang for tweet in answer],"likes": [tweet.favorite_count for tweet in answer],"retweets": [tweet.retweet_count for tweet in answer], "datetime":[tweet.created_at for tweet in answer]})
    df = df[df.language == "en"]
    df["Subjectivity"] = df["tweet"].apply(getSubjectivity)
    df["Polarity"] = df["tweet"].apply(getPolarity)
    df["Analysis"] = df["Polarity"].apply(getAnalysis)
    return df

In [12]:
paytm_csv = get_new_table("paytm")
paytm_csv.to_csv("paytm.csv", encoding="utf-8")

In [7]:
stocklist = ["ASIANPAINTS", 'AXIS BANK', 'BAJAJ AUTO','BAJAJ FINANCE', 'BAJAJ FINSERV', 'BHARTI AIRTEL', 'HCL TECHNOLOGIES', 'HDFC', 'HDFC BANK', 'HUL', 'ICICI BANK', 'INDUSIND BANK', 'INFOSYS', 'ITC', 'KOTAK MAHINDRA BANK', 'L&T','M3M','MARUTI SUZUKI', 'NESTLE', 'NTPC', 'ONGC', 'POWER GRID', 'RELIANCE IND', 'SBI', 'SUN PHARMA', 'TATA STEEL', 'TCS', 'TECH MAHINDRA', 'TITAN', 'ULTRATECH CEMENT']

In [8]:
stocklist2 = ["ASIANPAINTS", 'AXIS_BANK', 'BAJAJ_AUTO','BAJAJ_FINANCE', 'BAJAJ_FINSERV', 'BHARTI_AIRTEL', 'HCL_TECHNOLOGIES', 'HDFC', 'HDFC_BANK', 'HUL', 'ICICI_BANK', 'INDUSIND_BANK', 'INFOSYS', 'ITC', 'KOTAK_MAHINDRA_BANK', 'L&T','3M','MARUTI_SUZUKI', 'NESTLE', 'NTPC', 'ONGC', 'POWER_GRID', 'RELIANCE_IND', 'SBI', 'SUN_PHARMA', 'TATA_STEEL', 'TCS', 'TECH_MAHINDRA', 'TITAN', 'ULTRATECH_CEMENT']

In [6]:
stocklist_df = pd.DataFrame(stocklist, columns=["Symbol"])

In [10]:
stocklist_df = pd.DataFrame(stocklist2, columns=["Symbol"])

In [11]:
stocklist_df.to_csv("Stocklist2.csv", encoding="utf-8")

In [7]:
stocklist_df.to_csv("Stocklist.csv", encoding="utf-8")

In [27]:
stocklist[16]

'M3M'

In [8]:
def create_table_in_folder(stockname):
    csv = get_new_table(stockname)
    csv.to_csv("StockCSV/{}.csv".format(stockname), encoding="utf-8")

def add_in_table(stockname):
    csv = get_new_table(stockname)
    csv.to_csv("StockCSV/{}.csv".format(stockname),mode = 'a',header = False, encoding="utf-8")

In [71]:
add_in_table(stocklist[29])

In [9]:
tata_df = get_new_table("TATASTEEL")
tata_df.to_csv('tata.csv', mode='a', index=False, header=False)

In [1]:
import sched, time

In [ ]:
import requests

In [2]:
s = sched.scheduler(time.time, time.sleep)

In [4]:
def add_in_table(): 
    time.sleep(60*60*6)
    for stock in stocklist:
        add_in_table(stock)
    return add_in_table()
    

In [9]:
add_in_table()

TypeError: add_in_table() missing 1 required positional argument: 'stockname'

In [9]:
for i in stocklist:
    add_in_table(i)

In [4]:
df_here = pd.read_csv("StockCSV/ASIANPAINTS.csv")
df_here["Date"] = pd.to_datetime(df_here["datetime"])

In [5]:
df_here["Date"].unique()

<DatetimeArray>
['2021-11-16 20:31:57+00:00', '2021-11-16 19:19:23+00:00',
 '2021-11-16 19:13:10+00:00', '2021-11-16 19:03:21+00:00',
 '2021-11-16 18:54:58+00:00', '2021-11-16 18:52:38+00:00',
 '2021-11-16 18:47:30+00:00', '2021-11-16 18:29:13+00:00',
 '2021-11-16 18:24:30+00:00', '2021-11-16 18:14:20+00:00',
 ...
 '2021-12-09 19:16:55+00:00', '2021-12-09 19:10:22+00:00',
 '2021-12-09 18:49:49+00:00', '2021-12-09 18:41:01+00:00',
 '2021-12-09 18:38:49+00:00', '2021-12-09 18:34:20+00:00',
 '2021-12-09 18:10:40+00:00', '2021-12-09 17:54:04+00:00',
 '2021-12-09 17:41:11+00:00', '2021-12-09 17:23:46+00:00']
Length: 968, dtype: datetime64[ns, UTC]